## **Module**

In [1]:
from wmbio import * 

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Best Sub-Group Selection**

* **FILE PATH**

In [2]:
os.chdir("..")
os.getcwd()

'/home/wmbio/WORK/gitworking/Multi-omics-intergration'

In [3]:
# PATH
CANCER_TYPE = "LIHC"
GROUP_PHTH = os.getcwd() + '/group/'
PNG_PATH = os.getcwd() + '/png/'
GROUP_VALIDATION_PATH = os.getcwd() + '/group_validation/'
DEG_PATH = os.getcwd() + "/best_deg/"
RDATA_PATH = os.getcwd() + "/RAW_DATA/GDC_PREPARE/"
METHOD = 'deseq2'

In [4]:
# Load Validation score
col=['FILENAME','Log Rank Test','Silhouette','RNA_ANOVA_F1','RNA_RF_F1',
     'miRNA_ANOVA_F1','miRNA_RF_F1','Methylation_ANOVA_F1','Methylation_RF_F1']

group_score = pd.read_csv(GROUP_VALIDATION_PATH + 'test_validation.csv', usecols=col)

# Condition for Filtering
filter_cond = (group_score['Silhouette'] >= 0.2) & (group_score['Log Rank Test'] < 0.05) & \
              ((group_score['RNA_ANOVA_F1'] > 80) | (group_score['RNA_RF_F1'] > 80)) & \
              ((group_score['miRNA_ANOVA_F1'] > 80) | (group_score['miRNA_RF_F1'] > 80)) & \
              ((group_score['Methylation_ANOVA_F1'] > 85) | (group_score['Methylation_RF_F1'] > 85))
group_score = group_score[filter_cond].sort_values(["Silhouette"], ascending = (False))

bestSubgroup = group_score.FILENAME.to_list()

# 5 fold로 제한.
if len(bestSubgroup) > 10:
    bestSubgroup = bestSubgroup[:10]

# DEA
dea_result = list()
for best_group in bestSubgroup:
    SAMPLE_GROUP = GROUP_PHTH + CANCER_TYPE + "_GROUP_" + best_group + ".txt"
    
    # DEG Extraction
    deg_list = deg_extract(log_fc=1.5, fdr=0.05,
                   cancer_type=CANCER_TYPE, 
                   sample_group=SAMPLE_GROUP, deg_path=DEG_PATH, 
                   file_name=best_group,
                   rdata_path=RDATA_PATH,
                   method=METHOD)
    
    dea_result.append(deg_list)

# Filter DEA
# combine result
if METHOD == 'all':
    dea_combine = list(map(deseq2_edger_combine, dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'outer'), dea_combine)
elif METHOD == 'deseq2' :
    dea_combine = list(map(lambda d : d[["row", "log2FoldChange"]], dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'inner'), dea_combine)

# median & mean
dea_combine["SubGroup-log2FC_median"] = dea_combine.iloc[:, 1:].median(axis=1)
dea_combine["SubGroup-log2FC_mean"] = dea_combine.iloc[:, 1:].mean(axis=1)